## Week 3 Part 2
#### Build a dataframe of the postal code of each neighborhood along with borough name and neighborhood name in Toronto
#### Get the Geographical coordinates of the neighborhoods in toronto
#### Clustering the neighborhoods

In [1]:
!pip install folium
!pip install geopy

In [2]:
import numpy as np
import pandas as pd 
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
import json
from geopy.geocoders import Nominatim
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

### scraping data from wikipedia and storing in a dataframe

In [3]:
data=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text # sending GET request

In [4]:
bs=BeautifulSoup(data,'html.parser') #parsing data from html

In [5]:
PostalCodeList=[]
BoroughList=[]
NeighborhoodList=[]

In [6]:
for row in bs.find('table').find_all('tr'):  #finding all the rows in the table and append them into the lists
    cells=row.find_all('td')
    if(len(cells)>0):
        PostalCodeList.append(cells[0].text)
        BoroughList.append(cells[1].text)
        NeighborhoodList.append(cells[2].text.rstrip('\n')) # stripping new lines in the neighborhood cell

In [7]:
Toronto_df=pd.DataFrame({"PostalCode": PostalCodeList, "Borough":BoroughList, "Neighborhood": NeighborhoodList})  #creating a new dataframe with the lists
Toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Dropping the "Not assigned" cells in borough

In [8]:
Toronto_df_dropna=Toronto_df[Toronto_df.Borough!="Not assigned"].reset_index(drop=True)
Toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


### Grouping neighborhoods in the same borough


In [9]:
Toronto_df_grouped=Toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
Toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Make the value same as Borough for Neighborhood="Not assigned"


In [10]:
for index, row in Toronto_df_grouped.iterrows():
    if row["Neighborhood"]=="Not assigned":
        row["Neighborhood"]=row["Borough"]
Toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Checking whether the dataframe is same as shown in the question

In [11]:
column=["PostalCode", "Borough", "Neighborhood"]
test_df=pd.DataFrame(columns=column)
test_list=["M5G","M2H","M4B","M1J","M4G","M4M","M1R","M9V","M9L","M5V","M1B","M5A"]
for postcode in test_list:
    test_df=test_df.append(Toronto_df_grouped[Toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens, Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale, Wexford"
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


### Print the number of rows of the cleaned dataframe

In [12]:
Toronto_df_grouped.shape

(103, 3)

### Load the Coordinates from the csv file

In [13]:
coordinates=pd.read_csv('http://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
coordinates.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Combining 2 tables for coordinates


In [15]:
Toronto_df_new=Toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
Toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Check whether coordinates are added 

In [16]:
column_names=["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df=pd.DataFrame(columns=column_names)
test_list=["M5G","M2H","M4B","M1J","M4G","M4M","M1R","M9V","M9L","M5V","M1B","M5A"]
for postcode in test_list:
    test_df=test_df.append(Toronto_df_new[Toronto_df_new["PostalCode"]==postcode],ignore_index=True)
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


### Geopy to find Latitude and Longitude of Toronto

In [17]:
address= 'Toronto'
geolocator=Nominatim(user_agent="my-application")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographical coordinates of Toronto are {}, {}.' .format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207.


### Creating a map of Toronto along with its neighborhoods

In [18]:
map_toronto=folium.Map(location=[latitude, longitude], zoom_start=10)
for LAT, LON, Borough, Neighborhood in zip(Toronto_df_new['Latitude'], Toronto_df_new['Longitude'], Toronto_df_new['Borough'], Toronto_df_new['Longitude']):
    label='{}, {}'.format(Neighborhood, Borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker([LAT, LON], radius=5, popup=label, color='blue', fill=True, fill_color='blue', fill_opacity=0.5).add_to(map_toronto)
map_toronto

### Filtering boroughs with the word Toronto

In [19]:
Borough_names=list(Toronto_df_new.Borough.unique())  #filtering names
Borough_Toronto=[]
for x in Borough_names:
    if "toronto" in x.lower():
        Borough_Toronto.append(x)

Borough_Toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [20]:
Toronto_df_new=Toronto_df_new[Toronto_df_new['Borough'].isin(Borough_Toronto)].reset_index(drop=True) # creating a new dataframe with boroughs with word Toronto
print(Toronto_df_new.shape)
Toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [21]:
map_toronto=folium.Map(location=[latitude,longitude], zoom_start=10)
for LAT, LON, Borough, Neighborhood in zip(Toronto_df_new['Latitude'], Toronto_df_new['Longitude'], Toronto_df_new['Borough'], Toronto_df_new['Longitude']):
    label='{}, {}'.format(Neighborhood, Borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker([LAT, LON], radius=5, popup=label, color='blue', fill=True, fill_color='blue', fill_opacity=0.5).add_to(map_toronto)
map_toronto

### Using Foursquare

In [22]:
CLIENT_ID='GFWHTGXXRVLI0XDCVZKWO0U2UI35R3RGEFBRFROZPHYKC5H2'
CLIENT_SECRET='ZU4AEP3IRF03PZ4NKPDBJNY3AYCNL1LAMIF0V4EVIEHFKR0U'
VERSION='20180605'

print('Credential:')
print('Client ID:' +CLIENT_ID)
print('CLIENT_SECRET:' +CLIENT_SECRET)

Credential:
Client ID:GFWHTGXXRVLI0XDCVZKWO0U2UI35R3RGEFBRFROZPHYKC5H2
CLIENT_SECRET:ZU4AEP3IRF03PZ4NKPDBJNY3AYCNL1LAMIF0V4EVIEHFKR0U


In [23]:
radius=500
LIMIT=100
venues=[]
for LAT, LON, Post, Borough, Neighborhood in zip(Toronto_df_new['Latitude'], Toronto_df_new['Longitude'], Toronto_df_new['PostalCode'], Toronto_df_new['Borough'], 
                                                 Toronto_df_new['Neighborhood']):
    url="https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID, CLIENT_SECRET, VERSION, LAT, LON, radius, LIMIT)
    results=requests.get(url).json()["response"]['groups'][0]['items']
    for venue in results:
        venues.append(( 
            Post, Borough, Neighborhood, LAT, LON,
            venue['venue']['name'], venue['venue']['location']['lat'], venue['venue']['location']['lng'], venue['venue']['categories'][0]['name']))

In [24]:
#converting the venue list into dataframe
venues_df=pd.DataFrame(venues)
venues_df.columns=['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude','BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1713, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [47]:
#checking how many venues weree returned for each postal code
venues_df.groupby(["PostalCode","Borough","Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West, Riverdale",42,42,42,42,42,42
M4L,East Toronto,"The Beaches West, India Bazaar",19,19,19,19,19,19
M4M,East Toronto,Studio District,42,42,42,42,42,42
M4N,Central Toronto,Lawrence Park,4,4,4,4,4,4
M4P,Central Toronto,Davisville North,7,7,7,7,7,7
M4R,Central Toronto,North Toronto West,21,21,21,21,21,21
M4S,Central Toronto,Davisville,34,34,34,34,34,34
M4T,Central Toronto,"Moore Park, Summerhill East",4,4,4,4,4,4


In [25]:
# finding unique categories
print('{} Unique Categories'.format(len(venues_df['VenueCategory'].unique())))

230 Unique Categories


In [26]:
venues_df['VenueCategory'].unique()[:50]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Greek Restaurant', 'Cosmetics Shop', 'Italian Restaurant',
       'Ice Cream Shop', 'Brewery', 'Yoga Studio',
       'Fruit & Vegetable Store', 'Dessert Shop', 'Pizza Place',
       'Bookstore', 'Restaurant', 'Juice Bar', 'Bubble Tea Shop', 'Diner',
       'Spa', 'Furniture / Home Store', 'Grocery Store', 'Coffee Shop',
       'Bakery', 'Caribbean Restaurant', 'Frozen Yogurt Shop',
       'American Restaurant', 'Liquor Store', 'Gym', 'Burger Joint',
       'Fish & Chips Shop', 'Park', 'Sushi Restaurant', 'Burrito Place',
       'Pet Store', 'Steakhouse', 'Fast Food Restaurant', 'Movie Theater',
       'Sandwich Place', 'Food & Drink Shop', 'Fish Market', 'Café',
       'Cheese Shop', 'Gay Bar', 'Seafood Restaurant',
       'Middle Eastern Restaurant', 'Comfort Food Restaurant',
       'Stationery Store', 'Thai Restaurant', 'Wine Bar',
       'Coworking Space'], dtype=object)

### Analyze each area

In [31]:
Toronto_enc=pd.get_dummies(venues_df[['VenueCategory']], prefix=" ", prefix_sep="") #one got encoding
Toronto_enc['PostalCode'] = venues_df['PostalCode']
Toronto_enc['Borough']=venues_df['Borough']
Toronto_enc['Neighborhoods'] = venues_df['Neighborhood']
fixed_columns=list(Toronto_enc.columns[-3:])+list(Toronto_enc.columns[:-3]) # moving postal, borough and Neighborhoods column to first column
Toronto_enc = Toronto_enc[fixed_columns]
print(Toronto_enc.shape)
Toronto_enc.head()

(1713, 233)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [32]:
Toronto_grouped=Toronto_enc.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()
print(Toronto_grouped.shape)
Toronto_grouped

(39, 233)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [38]:
num_top_venues=10
indicators=['st','nd','rd']   #creating columns according to the number of top venues
areacolumns=['PostalCode', 'Borough', 'Neighborhoods']
topcolumns=[]
for ind in np.arange(num_top_venues):
    try:
        topcolumns.append('{}{} Most Common Venue' .format(ind+1, indicators[ind]))
    except:
        topcolumns.append('{}th Most Common Venue' .format(ind+1))
columns=areacolumns+topcolumns
Neighborhoods_venues_sorted=pd.DataFrame(columns=columns)
Neighborhoods_venues_sorted['PostalCode']=Toronto_grouped['PostalCode']
Neighborhoods_venues_sorted['Borough']=Toronto_grouped['Borough']
Neighborhoods_venues_sorted['Neighborhoods']=Toronto_grouped['Neighborhoods']
for ind in np.arange(Toronto_grouped.shape[0]):
    row_categories=Toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    Neighborhoods_venues_sorted.iloc[ind, 3:]=row_categories_sorted.index.values[0:num_top_venues]
print(Neighborhoods_venues_sorted.shape)
Neighborhoods_venues_sorted



(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Trail,Neighborhood,Pub,Health Food Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Yoga Studio,Dance Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Restaurant,Furniture / Home Store,Ice Cream Shop,Caribbean Restaurant,Pub,Bakery
2,M4L,East Toronto,"The Beaches West, India Bazaar",Sandwich Place,Park,Pub,Food & Drink Shop,Liquor Store,Burger Joint,Burrito Place,Italian Restaurant,Fast Food Restaurant,Ice Cream Shop
3,M4M,East Toronto,Studio District,Café,Coffee Shop,American Restaurant,Italian Restaurant,Brewery,Bakery,Gastropub,Yoga Studio,Pet Store,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,Park,Construction & Landscaping,Bus Line,Swim School,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
5,M4P,Central Toronto,Davisville North,Park,Hotel,Breakfast Spot,Gym,Sandwich Place,Food & Drink Shop,Department Store,Dumpling Restaurant,Donut Shop,Doner Restaurant
6,M4R,Central Toronto,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Salon / Barbershop,Restaurant,Café,Chinese Restaurant,Pet Store,Cosmetics Shop,Sporting Goods Shop
7,M4S,Central Toronto,Davisville,Sandwich Place,Pizza Place,Dessert Shop,Sushi Restaurant,Gym,Café,Italian Restaurant,Coffee Shop,Park,Brewery
8,M4T,Central Toronto,"Moore Park, Summerhill East",Park,Playground,Tennis Court,Trail,Donut Shop,Doner Restaurant,Dumpling Restaurant,Dog Run,Cupcake Shop,Discount Store
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Coffee Shop,Pub,Supermarket,Sports Bar,Sushi Restaurant,Vietnamese Restaurant,Light Rail Station,Liquor Store,American Restaurant,Pizza Place


### Cluster Areas 
Clustering Toronto into 5 clusters

In [41]:
k=5
Toronto_grouped_clustering=Toronto_grouped.drop(["PostalCode","Borough","Neighborhoods"], 1)
kmeans=KMeans(n_clusters=k, random_state=0).fit(Toronto_grouped_clustering)
kmeans.labels_[0:10]

array([0, 1, 1, 1, 1, 1, 1, 1, 3, 1], dtype=int32)

In [44]:
#creating a new dataframe with clusters as well as the top 10 venues
Toronto_merged=Toronto_df_new.copy()
Toronto_merged["Cluster Labels"]=kmeans.labels_
Toronto_merged=Toronto_merged.join(Neighborhoods_venues_sorted.drop(["Borough","Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")
print(Toronto_merged.shape)
Toronto_merged.head()

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Neighborhood,Pub,Health Food Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Yoga Studio,Dance Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Restaurant,Furniture / Home Store,Ice Cream Shop,Caribbean Restaurant,Pub,Bakery
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Sandwich Place,Park,Pub,Food & Drink Shop,Liquor Store,Burger Joint,Burrito Place,Italian Restaurant,Fast Food Restaurant,Ice Cream Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,American Restaurant,Italian Restaurant,Brewery,Bakery,Gastropub,Yoga Studio,Pet Store,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Construction & Landscaping,Bus Line,Swim School,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [47]:
#sort results by cluster labels
print(Toronto_merged.shape)
Toronto_merged.sort_values(["Cluster Labels"], inplace=True)
Toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Neighborhood,Pub,Health Food Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Yoga Studio,Dance Studio
20,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,1,Coffee Shop,Café,Hotel,Restaurant,Seafood Restaurant,Gastropub,Bar,Japanese Restaurant,Italian Restaurant,Deli / Bodega
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,1,Coffee Shop,Café,Restaurant,Hotel,Seafood Restaurant,American Restaurant,Bakery,Italian Restaurant,Gastropub,Steakhouse
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,1,Sandwich Place,Café,Coffee Shop,BBQ Joint,Pharmacy,Middle Eastern Restaurant,Indian Restaurant,History Museum,Plaza,Pizza Place
25,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049,1,Café,Bookstore,Bar,Bakery,Japanese Restaurant,Restaurant,Beer Bar,Beer Store,Sandwich Place,Pub
26,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,1,Vietnamese Restaurant,Café,Bar,Chinese Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Mexican Restaurant,Cocktail Bar,Dessert Shop
27,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,1,Airport Service,Airport Lounge,Airport Terminal,Airport,Airport Food Court,Airport Gate,Bar,Boutique,Harbor / Marina,Boat or Ferry
28,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,1,Coffee Shop,Café,Cocktail Bar,Seafood Restaurant,Restaurant,Italian Restaurant,Beer Bar,Hotel,Japanese Restaurant,Art Gallery
29,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,1,Coffee Shop,Café,Restaurant,Steakhouse,Gastropub,Gym,Bar,Japanese Restaurant,Burger Joint,Asian Restaurant
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Athletics & Sports,Restaurant,Baby Store,Gas Station,Nightclub,Coffee Shop,Candy Store


###  visualize the clusters

In [51]:
map_clusters=folium.Map(location=[latitude, longitude], zoom_start=11)
x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]
markers_colors=[]
for lat, lon, post, bor, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['PostalCode'], Toronto_merged['Borough'],
                                            Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label=folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi,cluster), parse_html=True)
    folium.CircleMarker(
    [lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.6).add_to(map_clusters)
map_clusters

### Examine Clusters

#### Cluster 1

In [53]:
Toronto_merged.loc[Toronto_merged['Cluster Labels']==0, Toronto_merged.columns[[1]+ list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Trail,Neighborhood,Pub,Health Food Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Yoga Studio,Dance Studio


#### Cluster 2

In [54]:
Toronto_merged.loc[Toronto_merged['Cluster Labels']==1, Toronto_merged.columns[[1]+ list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Seafood Restaurant,Gastropub,Bar,Japanese Restaurant,Italian Restaurant,Deli / Bodega
21,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Seafood Restaurant,American Restaurant,Bakery,Italian Restaurant,Gastropub,Steakhouse
24,Central Toronto,1,Sandwich Place,Café,Coffee Shop,BBQ Joint,Pharmacy,Middle Eastern Restaurant,Indian Restaurant,History Museum,Plaza,Pizza Place
25,Downtown Toronto,1,Café,Bookstore,Bar,Bakery,Japanese Restaurant,Restaurant,Beer Bar,Beer Store,Sandwich Place,Pub
26,Downtown Toronto,1,Vietnamese Restaurant,Café,Bar,Chinese Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Mexican Restaurant,Cocktail Bar,Dessert Shop
27,Downtown Toronto,1,Airport Service,Airport Lounge,Airport Terminal,Airport,Airport Food Court,Airport Gate,Bar,Boutique,Harbor / Marina,Boat or Ferry
28,Downtown Toronto,1,Coffee Shop,Café,Cocktail Bar,Seafood Restaurant,Restaurant,Italian Restaurant,Beer Bar,Hotel,Japanese Restaurant,Art Gallery
29,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Steakhouse,Gastropub,Gym,Bar,Japanese Restaurant,Burger Joint,Asian Restaurant
30,Downtown Toronto,1,Grocery Store,Café,Park,Athletics & Sports,Restaurant,Baby Store,Gas Station,Nightclub,Coffee Shop,Candy Store
31,West Toronto,1,Pharmacy,Bakery,Park,Fast Food Restaurant,Café,Recording Studio,Bar,Bank,Supermarket,Middle Eastern Restaurant


#### Cluster 3

In [55]:
Toronto_merged.loc[Toronto_merged['Cluster Labels']==2, Toronto_merged.columns[[1]+ list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,2,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 4

In [56]:
Toronto_merged.loc[Toronto_merged['Cluster Labels']==3, Toronto_merged.columns[[1]+ list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,3,Park,Playground,Tennis Court,Trail,Donut Shop,Doner Restaurant,Dumpling Restaurant,Dog Run,Cupcake Shop,Discount Store
10,Downtown Toronto,3,Park,Playground,Trail,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


#### Cluster 5

In [57]:
Toronto_merged.loc[Toronto_merged['Cluster Labels']==4, Toronto_merged.columns[[1]+ list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,4,Garden,Music Venue,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


### Observations
Most of the neighborhoods fall into cluster 2 which are mostly business areas with cafe, restaurants, supermarkets etc. Cluster 1 is a mostly trail and Neighborhood, Cluster 3 is  Cluster 5 is garden